# [Fonte](https://www.kaggle.com/code/jhoward/how-does-a-neural-net-really-work)

In [ ]:
from ipywidgets import interact
from fastai.basics import *

plt.rc('figure', dpi=90)

def plot_function(f, title=None, min=-2.1, max=2.1, color='r', ylim=None):
    x = torch.linspace(min,max, 100)[:,None]
    if ylim: plt.ylim(ylim)
    plt.plot(x, f(x), color)
    if title is not None: plt.title(title)

Essa é a noção função Quadrática, de coeficientes 3, 2 e 1.

In [ ]:
def f(x): return 3*x**2 + 2*x + 1

plot_function(f, "$3x^2 + 2x + 1$")

Agora imagine que não sabemos qual equação gerou ela, originalmente.

In [ ]:
# Gerar função quadrática
def quad(a, b, c, x):
  return a*x**2 + b*x + c

# Padronizar criação da equação quadrática
def mk_quad(a,b,c):
  return partial(quad, a,b,c) # <- dúvida

In [ ]:
f2 = mk_quad(3,2,1)
plot_function(f2)

Agora vamos imaginar uma situação em que a equação original não é conhecida e o nosso objetivo é apróximar o máximo possível dela.

In [ ]:
def noise(x, scale):
  return np.random.normal(scale=scale, size=x.shape)

def add_noise(x, mult, add):
  return x * (1+noise(x,mult)) + noise(x,add)

In [ ]:
np.random.seed(42)

x = torch.linspace(-2, 2, steps=20)[:,None]
y = add_noise(f(x), 0.15, 1.5)

plt.scatter(x,y);

O objetivo desta função é que manualmente, interagindo com os coeficientes, a, b e c você possa encontrar a melhor curva possível que compreendo os pontos (como na nossa função original).

In [ ]:
@interact(a=1.1, b=1.1, c=1.1)
def plot_quad(a, b, c):
    plt.scatter(x,y)
    plot_function(mk_quad(a,b,c), ylim=(-3,13))

Para que se possa ter ciência dos resultados, seria interessante alguma forma de medir o "erro" entre o nosso ajuste e os pontos. Nesse caso usaremos o MAE (MEAN SQUARE ERROR)


![](https://miro.medium.com/v2/resize:fit:723/1*9BhnZiaHkApC-gQt3rYpMQ.png)

In [ ]:
import matplotlib
%matplotlib inline

def mae(preds, acts):
  return (torch.abs(preds-acts)).mean()

@interact(a=1.1, b=1.1, c=1.1)
def plot_quad(a, b, c):
    f = mk_quad(a,b,c)
    plt.scatter(x,y)
    loss = mae(f(x), y)
    plot_function(f, ylim=(-3,12), title=f"MAE: {loss:.2f}")

Claro que seria inviável fazer isto automaticamente, portanto para automar podemos usar Pytorch que aplicando os conceitos do cálculo (Derivas em especial) consegue calcular esse erro, o nome disso é gradiente.

In [ ]:
def quad_mae(params):
    f = mk_quad(*params)
    return mae(f(x), y)

abc = torch.tensor([1.5,1.5,1.5])
abc.requires_grad_()

In [ ]:
loss = quad_mae(abc)
loss.backward()

abc.grad

In [ ]:
with torch.no_grad():
    abc -= abc.grad*0.01
    loss = quad_mae(abc)

print(f'loss={loss:.2f}')

In [ ]:
for i in range(10):
    loss = quad_mae(abc)
    loss.backward()
    with torch.no_grad(): abc -= abc.grad*0.01
    print(f'step={i}; loss={loss:.2f}')

Claro que, nem todas as equações no mundo serão quadátricas, logo precisamos de algo mais genérico.

In [ ]:
def rectified_linear(m,b,x):
    y = m*x+b
    return torch.clip(y, 0.)

plot_function(partial(rectified_linear, 1,1))

In [ ]:
@interact(m=1.5, b=1.5)
def plot_relu(m, b):
    plot_function(partial(rectified_linear, m,b), ylim=(-1,4))

Essa função (acima) não é tão interessante, mas podemos criar uma "double_relu", o que nos dá mais controle sobre a linha.

In [ ]:
def double_relu(m1,b1,m2,b2,x):
    return rectified_linear(m1,b1,x) + rectified_linear(m2,b2,x)

@interact(m1=-1.5, b1=-1.5, m2=1.5, b2=1.5)
def plot_double_relu(m1, b1, m2, b2):
    plot_function(partial(double_relu, m1,b1,m2,b2), ylim=(-1,6))

Podemos adicionar quantas "Relus" forem necessárias ou quisermos de modo que elas podem se adequar aos mais diversos conjunto de dados e seus padrões.